In [1]:
import keras
from keras import layers
from keras.datasets import mnist
import numpy as np

(train_data, _), (test_data, _) = mnist.load_data()
# Scale images to the [0, 1] range
train_data = train_data.astype("float32") / 255
test_data = test_data.astype("float32") / 255
# Make sure images have shape (28, 28, 1)

#train_data = np.expand_dims(train_data, -1)
#test_data = np.expand_dims(test_data, -1)

train_data = np.reshape(train_data, (len(train_data), 28, 28, 1))
test_data = np.reshape(test_data, (len(test_data), 28, 28, 1))

noise_strength = 0.6



def noise(array):
    """
    Adds random noise to each image in the supplied array.
    """
    noise_factor = 0.70
    noisy_array = array + noise_factor * np.random.normal(
        loc=0.0, scale=1.0, size=array.shape
    )

    return np.clip(noisy_array, 0.0, 1.0)


train_data_noisy = noise(train_data)
test_data_noisy = noise(test_data)

2022-11-26 16:06:35.758979: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-26 16:06:36.159795: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2022-11-26 16:06:36.224158: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-11-26 16:06:36.224181: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

In [2]:
input = layers.Input(shape=(28,28,1))
lnorm = layers.Normalization()
lnorm.adapt(data=train_data_noisy)
x = lnorm(input)
#Encoder
x = layers.Conv2D(32, (3, 3), padding="same", activation='relu')(x)
x = layers.MaxPooling2D()(x)
x = layers.Conv2D(64, (3, 3), padding="same", activation='relu')(x)
x = layers.MaxPooling2D()(x)
#espace latent
x = layers.Dense(2, activation=None)(x)
#decoder
x = layers.Conv2DTranspose(64, padding='same', activation='relu', strides=2, kernel_size=(3,3))(x)
x = layers.Conv2DTranspose(32, padding='same', activation='relu' ,strides=2, kernel_size=(3,3))(x)
x = layers.Conv2DTranspose(1, padding='same', activation=None, strides=1, kernel_size=(3,3))(x)


autoencoder = keras.Model(input, x)
autoencoder.compile(optimizer="adam", loss="mean_squared_error")
autoencoder.summary()

2022-11-26 16:06:43.646627: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-26 16:06:43.646916: W tensorflow/stream_executor/cuda/cuda_driver.cc:263] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-26 16:06:43.646959: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (mithrandir-laptop): /proc/driver/nvidia/version does not exist
2022-11-26 16:06:43.648060: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 28, 28, 1)]       0         
                                                                 
 normalization (Normalizatio  (None, 28, 28, 1)        3         
 n)                                                              
                                                                 
 conv2d (Conv2D)             (None, 28, 28, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 14, 14, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 7, 7, 64)         0     

In [3]:
autoencoder.fit(
    x=train_data_noisy,
    y=train_data,
    epochs=10,
    batch_size=32,
    shuffle=True,
    validation_data=(test_data_noisy, test_data),
)

Epoch 1/10
 126/1875 [=>............................] - ETA: 1:13 - loss: 0.0456

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt
def display(array1, array2, array3=None):
    """
    Displays ten random images from each one of the supplied arrays.
    """

    n = 10

    indices = np.random.randint(len(array1), size=n)
    images1 = array1[indices, :]
    images2 = array2[indices, :]

    plt.figure(figsize=(20, 4))

    for i, (image1, image2) in enumerate(zip(images1, images2)):
        ax = plt.subplot(3, n, i + 1)
        plt.imshow(image1.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

        ax = plt.subplot(3, n, i + 1 + n)
        plt.imshow(image2.reshape(28, 28))
        plt.gray()
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)

    if array3 is not None:
        images3 = array3[indices, :]
        for i, (image3) in enumerate(zip(images3)):
            ax = plt.subplot(3, n, i + 1 + n*2)
            plt.imshow(np.reshape(image3,(28, 28)))
            plt.gray()
            ax.get_xaxis().set_visible(False)
            ax.get_yaxis().set_visible(False)

    plt.show()

In [ ]:
predictions = autoencoder.predict(test_data_noisy)

In [ ]:
display(test_data_noisy, predictions)